In [1]:
import numpy as np
import pandas as pd
import os
import scipy
import mne 

from mne_bids import write_raw_bids, BIDSPath, make_dataset_description

# Data available and basic information of dataset
Data are available at 'https://doi.org/10.6084/m9.figshare.c.5006684', which inculdes two parts of dataset. 

The one is about data from 20 adults subjects, which were named as "Axxx"(x represents the index of subjects). The other is about data from 96 children subjects, which were named as "Cxxx".



# Features of this published dataset
The data was preprocessed by authors including checking error trials according to several standards, reliable components analysis(RCA) et al. For details, see scripts in 'https://osf.io/fkjt6/'.

The dataset was formated as '.mat' file whose data was stored in the nest pattern. We used 'scipy' and index for several times to untie the nest. Finally, we load data in the format of 'ndarray'. 

The dataset has two parts of variables. The one can be used to do further processing, such as triggers and EEG data. The other reflects various intermediate preprocessing steps by authors, such as EOG, BlockBadElectrodes et al.



# The logic of preprocess
The published data were formated as '.mat' file, which can not be analysised using MNE. What's more, making the format of all of the datasets in this project into BIDS can ease analysis later. So, we decided to make format into BIDS first. 



# The process of analysis
1️⃣ Path setting: 
1) we set 'ds_root' for the location of loaded dataset, which will make data loading more easily. 

2) we set 'fn_*'for certain dataset because every dataset has a specific naming format for each subjects. Last, we check whether we have folder to save preprocessed results. If not, make a certain folder.

2️⃣ Process data into BIDS: 
1) we excluded several trials for two reasons: 1) Author had defined the variable 'GoodTrial' which uses the value '0' and '1' to indicate the effect of trials. So we extract the trials whose value is '1'. 2) The variable 'trigger' implies the timestamps for some events during the experiment, like 'onsets', 'response'. We exclude another trials due to the lossing trigger for 'response' because the analysis later is response-locked analysis.

2) we made EEG data into 2-dimension whose shape is (n_channels, n_timestamps). Note that the unit of EEG data should change into µV, which is standard format of BIDS. 

3) we prepare the information about triggers according to BIDS. For the 2-dimension format of EEG data, the timestamps of trigger should be continious. The shape of trigger in BIDS should be (n_trials, 3). The first columm represents timestamps. The second column can be set as '0' because this one will not be analysised. The third column represents trigger id which indicates the certain events of the trigger.

4) we need to import channel name for each channel. However, for this dataset, we did not find a certain file to define for the channel name. After searching for the official website, we find a published paper which compared the difference between 10-10 and HydroCel. From this paper, we find the channel of Cpz, Cp1 and Cp2 in 55th, 37th and 87th respectively. But we just made channel name file by ourselves. All the channel named as 'Ex'(x represents the number of channel)

5) we use certain functions in MNE to format the data into BIDS.


In [2]:
# set different paths to load and save data
ds_root='/volumes/hyijie_psy/CPP/data_low-level_1/data' # path for raw data
tgt_dir='..'
# set file name to load data 
fn_first='Formatted_'
fn_last='.3_40_preproc'

# make folders to save result
preprocessData=os.path.join(tgt_dir,'data/preprocessData')
if not os.path.exists(preprocessData):
    os.makedirs(preprocessData)


In [4]:
# Preprocess the dataset and format it according to the BIDS

#load original behavior data
data_behavior = pd.read_csv('../data/behaviorData_low_1_raw.csv')
data_behavior = data_behavior[data_behavior.iloc[:, 1].astype(str).str.startswith('A')]

# prepare empty dataset for filtered data
data_behavior_filter = []

sub_dir=[f'A{i:03d}' for i in range(1,21)]
for sub in sub_dir:

    # load data for certain subjects
    data_behavior_perSubject = data_behavior[data_behavior.iloc[:,1] == sub]
    data_behavior_perSubject.iloc[:, 2] = data_behavior_perSubject.iloc[:, 2] - 1
    

    # load EEG data, corresponding triggers and length for each trial
    raw = scipy.io.loadmat(os.path.join(ds_root,fn_first+sub+fn_last+'.mat'))
    good_trials = raw['GoodTrial'][0][0][0].T
    data_EEG = raw['X']
    data_trigger = raw['Onsets']['corrMatrix'][0,0]
    trialEpoch_id = raw['trialEpochIdx'] # this represents the start and end timestamps for each trial
    
    ## exclude trials in two steps: 1) exclude bad trials; 2)exculde missing trials
    ## note that it is not necessary to remove eeg data because triggers were recorded in order
    # exclude bad trials according to author's analysis
    valid_trial = good_trials[:,0] == 1
    valid_trial_index = np.where(valid_trial)[0] # python style
    data_trigger_goodTrial = data_trigger[valid_trial_index,:]
    trialEpoch_id_goodTrial = trialEpoch_id[valid_trial_index,:]

    # filter behavior data according to good trials
    data_behavior_perSubject_goodTrial = data_behavior_perSubject[data_behavior_perSubject.iloc[:, 2].isin(valid_trial_index)]
    data_behavior_filter.append(data_behavior_perSubject_goodTrial)

    # exclude trials without triggers for response
    valid_row = ~np.isnan(data_trigger_goodTrial[:,4]) 
    index_validTrial = np.where(valid_row)[0]
    data_trigger_dropNaN = data_trigger_goodTrial[index_validTrial,:]
    trialEpoch_id_dropNaN = trialEpoch_id_goodTrial[index_validTrial,:]

    ## prepare for BIDS
    # combine eeg data into 2 dimensions. In addition, BIDS need µV as unit
    data_EEG_BIDS = np.concatenate(data_EEG[0], axis=1)
    data_EEG_BIDS = data_EEG_BIDS * 1e6 
    
    # make the trigger into continuous format and perform transposition
    # note: it is necessary to transfer trigger as continious format first
    data_trigger_continious = np.empty((data_trigger_dropNaN.shape[0],
                                        data_trigger_dropNaN.shape[1]),
                                        dtype = object)
    for trial in range(0,data_trigger_dropNaN.shape[0]): 
        data_trigger_continious[trial,:] = data_trigger_dropNaN[trial,:]+trialEpoch_id_dropNaN[trial,0]-1 # the reason for '-1' is for specific data format
    
    trigger_row = data_trigger_continious.shape[0]
    trigger_col = data_trigger_continious.shape[1]
    data_trigger_BIDS = np.empty((trigger_row * trigger_col , 3), # must be 3 for BIDS
                                 dtype = np.int32) # note that int32 is necessary
    for trial in range(0,trigger_row): 
        data_trigger_BIDS[trial*trigger_col : (trial+1)*trigger_col , 0] = data_trigger_continious[trial,:]-1 # note that, data format in matlab differs from that in python. That's why we minus 1 here
        data_trigger_BIDS[trial*trigger_col : (trial+1)*trigger_col , 1] = 0
        data_trigger_BIDS[trial*trigger_col : (trial+1)*trigger_col , 2] = np.arange(1,trigger_col+1)

    # sampling rate 
    samplingRate = 500
    
    # channel name, such as 'E1', 'E2' et al.
    channel_info = pd.read_csv('../data/channelName.csv')
    channel_info = channel_info.dropna()
    channelName = channel_info['channel'].tolist()
    
    # use mne_bids to save data 
    info = mne.create_info(ch_names=channelName, sfreq=samplingRate, ch_types='eeg')
    raw_mne = mne.io.RawArray(data_EEG_BIDS, info)

    bids_path = BIDSPath(
        subject=sub, 
        task='randomDot', 
        datatype='eeg',
        root=preprocessData
    )

    write_raw_bids(
        raw=raw_mne,
        bids_path=bids_path,
        events=data_trigger_BIDS,
        event_id={'fixation':1,
                  'boil':2,
                  'photodiode':3,
                  'stimulus':4, # note that the timings were corrected relative to the photodiode signal, so column 3 = column 4
                  'response':5, 
                  'offser':6}, 
        allow_preload=True,
        format='BrainVision',
        overwrite=True,
        verbose=True
    )

    make_dataset_description(
    path=preprocessData,
    name='LowLevel-randomDot',
    dataset_type='raw',
    overwrite=True
    )

data_behavior_filter_final = np.concatenate(data_behavior_filter, axis=0)
data_behavior_filter_final = pd.DataFrame(data_behavior_filter_final)
data_behavior_filter_final.columns = ['Pnumber', 'subj', 'trialno','group', 'age', 'sex', 'hands', 'devcond', 'coherence', 'motiondir', 'keypress', 'RT', 'accuracy'] 
data_behavior_filter_final.to_csv('../data/behaviorData_low_1_final_test.csv', index=False, header=True)

selected_columns = ['subj','coherence', 'RT', 'accuracy', 'keypress']
data_behavior_selected = data_behavior_filter_final[selected_columns]
data_behavior_selected.to_csv('../data/behaviorData_low_1_final.csv', index=False, header=True)



Creating RawArray with float64 data, n_channels=128, n_times=491753
    Range : 0 ... 491752 =      0.000 ...   983.504 secs
Ready.
Writing '../data/preprocessData/participants.tsv'...
Writing '../data/preprocessData/participants.json'...
Used Annotations descriptions: [np.str_('boil'), np.str_('fixation'), np.str_('offser'), np.str_('photodiode'), np.str_('response'), np.str_('stimulus')]
Writing '../data/preprocessData/sub-A001/eeg/sub-A001_task-randomDot_events.tsv'...
Writing '../data/preprocessData/sub-A001/eeg/sub-A001_task-randomDot_events.json'...
Writing '../data/preprocessData/dataset_description.json'...
Writing '../data/preprocessData/sub-A001/eeg/sub-A001_task-randomDot_eeg.json'...
Found no extension for raw file, assuming "BTi" format and appending extension .pdf
Writing '../data/preprocessData/sub-A001/eeg/sub-A001_task-randomDot_channels.tsv'...
Copying data files to sub-A001_task-randomDot_eeg.vhdr


/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_3337/2620168588.py:84: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_3337/2620168588.py:84: RuntimeWarning: Encountered data in "double" format. Converting to float32.
  write_raw_bids(


Writing '../data/preprocessData/sub-A001/sub-A001_scans.tsv'...
Wrote ../data/preprocessData/sub-A001/sub-A001_scans.tsv entry with eeg/sub-A001_task-randomDot_eeg.vhdr.
Writing '../data/preprocessData/dataset_description.json'...
Creating RawArray with float64 data, n_channels=128, n_times=461226
    Range : 0 ... 461225 =      0.000 ...   922.450 secs
Ready.
Writing '../data/preprocessData/participants.tsv'...
Writing '../data/preprocessData/participants.json'...
Used Annotations descriptions: [np.str_('boil'), np.str_('fixation'), np.str_('offser'), np.str_('photodiode'), np.str_('response'), np.str_('stimulus')]
Writing '../data/preprocessData/sub-A002/eeg/sub-A002_task-randomDot_events.tsv'...
Writing '../data/preprocessData/sub-A002/eeg/sub-A002_task-randomDot_events.json'...
Writing '../data/preprocessData/dataset_description.json'...
Writing '../data/preprocessData/sub-A002/eeg/sub-A002_task-randomDot_eeg.json'...
Found no extension for raw file, assuming "BTi" format and appen

/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_3337/2620168588.py:84: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_3337/2620168588.py:84: RuntimeWarning: Encountered data in "double" format. Converting to float32.
  write_raw_bids(


Writing '../data/preprocessData/sub-A002/sub-A002_scans.tsv'...
Wrote ../data/preprocessData/sub-A002/sub-A002_scans.tsv entry with eeg/sub-A002_task-randomDot_eeg.vhdr.
Writing '../data/preprocessData/dataset_description.json'...
Creating RawArray with float64 data, n_channels=128, n_times=449384
    Range : 0 ... 449383 =      0.000 ...   898.766 secs
Ready.
Writing '../data/preprocessData/participants.tsv'...
Writing '../data/preprocessData/participants.json'...
Used Annotations descriptions: [np.str_('boil'), np.str_('fixation'), np.str_('offser'), np.str_('photodiode'), np.str_('response'), np.str_('stimulus')]
Writing '../data/preprocessData/sub-A003/eeg/sub-A003_task-randomDot_events.tsv'...
Writing '../data/preprocessData/sub-A003/eeg/sub-A003_task-randomDot_events.json'...
Writing '../data/preprocessData/dataset_description.json'...
Writing '../data/preprocessData/sub-A003/eeg/sub-A003_task-randomDot_eeg.json'...
Found no extension for raw file, assuming "BTi" format and appen

...

Writing '../data/preprocessData/sub-A018/sub-A018_scans.tsv'...
Wrote ../data/preprocessData/sub-A018/sub-A018_scans.tsv entry with eeg/sub-A018_task-randomDot_eeg.vhdr.
Writing '../data/preprocessData/dataset_description.json'...
Creating RawArray with float64 data, n_channels=128, n_times=455674
    Range : 0 ... 455673 =      0.000 ...   911.346 secs
Ready.
Writing '../data/preprocessData/participants.tsv'...
Writing '../data/preprocessData/participants.json'...
Used Annotations descriptions: [np.str_('boil'), np.str_('fixation'), np.str_('offser'), np.str_('photodiode'), np.str_('response'), np.str_('stimulus')]
Writing '../data/preprocessData/sub-A019/eeg/sub-A019_task-randomDot_events.tsv'...
Writing '../data/preprocessData/sub-A019/eeg/sub-A019_task-randomDot_events.json'...
Writing '../data/preprocessData/dataset_description.json'...
Writing '../data/preprocessData/sub-A019/eeg/sub-A019_task-randomDot_eeg.json'...
Found no extension for raw file, assuming "BTi" format and appen

/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_3337/2620168588.py:84: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_3337/2620168588.py:84: RuntimeWarning: Encountered data in "double" format. Converting to float32.
  write_raw_bids(


Writing '../data/preprocessData/sub-A019/sub-A019_scans.tsv'...
Wrote ../data/preprocessData/sub-A019/sub-A019_scans.tsv entry with eeg/sub-A019_task-randomDot_eeg.vhdr.
Writing '../data/preprocessData/dataset_description.json'...
Creating RawArray with float64 data, n_channels=128, n_times=432717
    Range : 0 ... 432716 =      0.000 ...   865.432 secs
Ready.
Writing '../data/preprocessData/participants.tsv'...
Writing '../data/preprocessData/participants.json'...
Used Annotations descriptions: [np.str_('boil'), np.str_('fixation'), np.str_('offser'), np.str_('photodiode'), np.str_('response'), np.str_('stimulus')]
Writing '../data/preprocessData/sub-A020/eeg/sub-A020_task-randomDot_events.tsv'...
Writing '../data/preprocessData/sub-A020/eeg/sub-A020_task-randomDot_events.json'...
Writing '../data/preprocessData/dataset_description.json'...
Writing '../data/preprocessData/sub-A020/eeg/sub-A020_task-randomDot_eeg.json'...
Found no extension for raw file, assuming "BTi" format and appen

/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_3337/2620168588.py:84: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_3337/2620168588.py:84: RuntimeWarning: Encountered data in "double" format. Converting to float32.
  write_raw_bids(


Writing '../data/preprocessData/sub-A020/sub-A020_scans.tsv'...
Wrote ../data/preprocessData/sub-A020/sub-A020_scans.tsv entry with eeg/sub-A020_task-randomDot_eeg.vhdr.
Writing '../data/preprocessData/dataset_description.json'...
